In [1]:
import os
import json
import numpy as np
import torch
from model import create_toy_model

In [2]:
# 1) Load vocab and build one-hot tools
vocab_path = "vocab_chars.json"
with open(vocab_path, 'r') as f:
    vocab = json.load(f)

vocab_size = len(vocab)
print("Vocab size:", vocab_size)

# smiles -> one-hot [seq_len, vocab_size]
def smiles_to_onehot(smiles: str, vocab: dict, max_len: int = 50):
    one_hot = np.zeros((max_len, vocab_size), dtype=np.float32)
    for i, ch in enumerate(smiles[:max_len]):
        idx = vocab.get(ch, vocab.get('<unk>', 3))
        one_hot[i, idx] = 1.0
    # pad remainder with <pad>
    pad_id = vocab.get('<pad>', 0)
    for i in range(len(smiles), max_len):
        one_hot[i, pad_id] = 1.0
    return one_hot

Vocab size: 101


In [3]:
# 2) Create model and run a quick forward
model = create_toy_model(vocab_size=vocab_size)
model.eval()

seq_len = 50
smiles = "CC[NH+](CC)[C@](C)(CC)[C@H](O)c1cscc1Br"
one_hot = smiles_to_onehot(smiles, vocab, max_len=seq_len)
input_tensor = torch.from_numpy(one_hot).unsqueeze(0)  # [1, seq_len, vocab_size]

with torch.no_grad():
    out = model(input_tensor)

print("Input:", input_tensor.shape)
print("Output:", out.shape, out.item())

Input: torch.Size([1, 50, 101])
Output: torch.Size([1, 1]) 0.20094776153564453


In [4]:
# 3) Export TorchScript (.pt)
dummy = torch.zeros(1, seq_len, vocab_size, dtype=torch.float32)
traced = torch.jit.trace(model, dummy, strict=False, check_trace=False)
traced_path = "smiles_transformer_regression.pt"
traced.save(traced_path)

In [5]:
# 4) Save example input (.npy)
example_path = "example_input.npy"
np.save(example_path, input_tensor.numpy())
print("Saved example input:", example_path, input_tensor.numpy().shape)

Saved example input: example_input.npy (1, 50, 101)


In [6]:
# 5) Print Triton config info and save config.pbtxt
model_name = "smiles_transformer_regression"
platform = "pytorch_libtorch"
max_batch_size = 8
input_name = "input__0"
output_name = "output__0"

# Gather shapes from prior cells
seq_len = int(seq_len)
vocab_size = int(vocab_size)
input_dims = [seq_len, vocab_size]
output_dims = [1]

print("Triton config info:")
print({
    "name": model_name,
    "platform": platform,
    "max_batch_size": max_batch_size,
    "input": {"name": input_name, "dtype": "TYPE_FP32", "dims": input_dims},
    "output": {"name": output_name, "dtype": "TYPE_FP32", "dims": output_dims},
})

Triton config info:
{'name': 'smiles_transformer_regression', 'platform': 'pytorch_libtorch', 'max_batch_size': 8, 'input': {'name': 'input__0', 'dtype': 'TYPE_FP32', 'dims': [50, 101]}, 'output': {'name': 'output__0', 'dtype': 'TYPE_FP32', 'dims': [1]}}


In [7]:
config_content = f"""name: \"{model_name}\"
platform: \"{platform}\"
max_batch_size: {max_batch_size}

input [
  {{
    name: \"{input_name}\"
    data_type: TYPE_FP32
    dims: [{input_dims[0]}, {input_dims[1]}]
  }}
]

output [
  {{
    name: \"{output_name}\"
    data_type: TYPE_FP32
    dims: [{output_dims[0]}]
  }}
]

instance_group [
  {{
    count: 1
    kind: KIND_CPU
  }}
]

dynamic_batching {{
  preferred_batch_size: [1, 2, 4, 8]
  max_queue_delay_microseconds: 100
}}
"""

In [8]:
# Save config.pbtxt one level up so it matches Triton model repo layout
config_path = os.path.join("config.pbtxt")
with open(config_path, "w") as f:
    f.write(config_content)
print("Saved config.pbtxt:", config_path)

Saved config.pbtxt: config.pbtxt
